# GWExPy TimeSeries Interoperability Tutorial

This notebook introduces the new interoperability features added to the `gwexpy` `TimeSeries` class.
`gwexpy` can seamlessly convert data to and from popular data science libraries such as Pandas, Xarray, PyTorch, and Astropy.

## Table of Contents

- 1. General Data Formats & Data Infrastructure
    - 1.1. NumPy [Original GWpy]
    - 1.2. Pandas Integration [Original GWpy]
    - 1.3. Polars [GWExPy New]
    - 1.4. Xarray Integration [Original GWpy]
    - 1.5. Dask Integration [GWExPy New]
    - 1.6. JSON / Python Dict (to_json) [GWExPy New]
- 2. Database & Storage Layer
    - 2.1. HDF5 [Original GWpy]
    - 2.2. SQLite [GWExPy New]
    - 2.3. Zarr [GWExPy New]
    - 2.4. netCDF4 [GWExPy New]
- 3. Computer Science, Machine Learning & Accelerated Computing
    - 3.1. PyTorch Integration [GWExPy New]
    - 3.2. CuPy (CUDA Acceleration) [GWExPy New]
    - 3.3. TensorFlow Integration [GWExPy New]
    - 3.4. JAX Integration [GWExPy New]
- 4. Astronomy & Gravitational Wave Physics
    - 4.1. PyCBC / LAL [Original GWpy]
    - 4.2. Astropy Integration [Original GWpy]
    - 4.3. Specutils Integration [GWExPy New]
    - 4.4. Pyspeckit Integration [GWExPy New]
- 5. Particle Physics & High Energy Physics
    - 5.1. CERN ROOT Integration [GWExPy New]
    - 5.2. Recovering from ROOT Objects (`from_root`)
- 6. Geophysics, Seismology & Electromagnetics
    - 6.1. ObsPy [Original GWpy]
    - 6.2. SimPEG Integration [GWExPy New]
    - 6.3. MTH5 / MTpy [GWExPy New]
- 7. Acoustics & Audio Analysis
    - 7.1. Librosa / Pydub [GWExPy New]
- 8. Medical & Biosignal Analysis
    - 8.1. MNE-Python [GWExPy New]
    - 8.2. Elephant / quantities Integration [GWExPy New]
    - 8.3. Neo [GWExPy New]
- 9. Summary

In [1]:
import os

os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")

'3'

In [2]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
from astropy import units as u
from gwpy.time import LIGOTimeGPS

from gwexpy.timeseries import TimeSeries

warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
warnings.filterwarnings("ignore", category=UserWarning)

/home/washimi/miniforge3/envs/ws-base/lib/python3.12/site-packages/gwpy/time/__init__.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  from lal import LIGOTimeGPS


In [3]:
# Create sample data
# Generate a 10-second, 100Hz sine wave
rate = 100 * u.Hz
dt = 1 / rate
t0 = LIGOTimeGPS(1234567890, 0)
duration = 10 * u.s
size = int(rate * duration)
times = np.arange(size) * dt.value
data = np.sin(2 * np.pi * 1.0 * times)  # 1Hz sine wave

ts = TimeSeries(data, t0=t0, dt=dt, unit="V", name="demo_signal")
print("Original TimeSeries:")
print(ts)
ts.plot(title="Original TimeSeries");

## 1. General Data Formats & Data Infrastructure

### 1.1. NumPy <span style="color: #3498db; font-weight: bold;">[Original GWpy]</span>

> **NumPy**: NumPy is the foundational library for numerical computing in Python, providing multidimensional arrays and fast mathematical operations.
> 📚 [NumPy Documentation](https://numpy.org/)

You can obtain NumPy arrays via `TimeSeries.value` or `np.asarray(ts)`.

### 1.2. Pandas Integration <span style="color: #3498db; font-weight: bold;">[Original GWpy]</span>

> **Pandas**: Pandas is a powerful library for data analysis and manipulation, providing flexible data structures through DataFrame and Series.
> 📚 [Pandas Documentation](https://pandas.pydata.org/)

Using the `to_pandas()` method, you can convert `TimeSeries` to `pandas.Series`.
The index can be selected from `datetime` (UTC), `gps`, or `seconds` (Unix timestamp).

In [4]:
try:
    # Convert to Pandas Series (default is datetime index)
    s_pd = ts.to_pandas(index="datetime")
    print("\n--- Converted to Pandas Series ---")
    display(s_pd)
    s_pd.plot(title="Pandas Series")
    plt.show()
    plt.close()

    # Restore TimeSeries from Pandas Series
    ts_restored = TimeSeries.from_pandas(s_pd, unit="V")
    print("\n--- Restored TimeSeries from Pandas ---")
    print(ts_restored)

    del s_pd, ts_restored

except ImportError:
    print("Pandas is not installed.")

### 1.3. Polars <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **Polars**: Polars is a high-performance DataFrame library implemented in Rust, excelling at large-scale data processing.
> 📚 [Polars Documentation](https://pola.rs/)

You can convert to Polars DataFrame/Series with `to_polars()`.

In [5]:
try:
    import polars as pl

    _ = pl
    # TimeSeries -> Polars DataFrame
    df_pl = ts.to_polars()
    print("--- Polars DataFrame ---")
    print(df_pl.head())

    # Plot using Polars/Matplotlib
    plt.figure()
    data_col = [c for c in df_pl.columns if c != "time"][0]
    plt.plot(df_pl["time"], df_pl[data_col])
    plt.title("Polars Data Plot")
    plt.show()

    # Recover to TimeSeries
    from gwexpy.timeseries import TimeSeries

    ts_recovered = TimeSeries.from_polars(df_pl)
    print("Recovered from Polars:", ts_recovered)

    del df_pl, ts_recovered

except ImportError:
    print("Polars not installed.")

### 1.4. Xarray Integration <span style="color: #3498db; font-weight: bold;">[Original GWpy]</span>

> **xarray**: xarray is a library for multidimensional labeled arrays, widely used for manipulating NetCDF data and analyzing meteorological and earth science data.
> 📚 [xarray Documentation](https://docs.xarray.dev/)

`xarray` is a powerful library for handling multidimensional labeled arrays. You can convert with `to_xarray()` while preserving metadata.

In [6]:
try:
    # Convert to Xarray DataArray
    da = ts.to_xarray()
    print("\n--- Converted to Xarray DataArray ---")
    print(da)
    # Verify metadata (attrs) is preserved
    print("Attributes:", da.attrs)

    da.plot()
    plt.title("Xarray DataArray")
    plt.show()
    plt.close()

    # Restore
    ts_x = TimeSeries.from_xarray(da)
    print("\n--- Restored TimeSeries from Xarray ---")
    print(ts_x)

    del da, ts_x

except ImportError:
    print("Xarray is not installed.")

### 1.5. Dask Integration <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **Dask**: Dask is a library for parallel computing that enables processing of large-scale data beyond NumPy/Pandas.
> 📚 [Dask Documentation](https://www.dask.org/)

In [7]:
try:
    import dask.array as da

    _ = da
    # Convert to Dask Array
    dask_arr = ts.to_dask(chunks="auto")
    print("\n--- Converted to Dask Array ---")
    print(dask_arr)

    # Restore (compute=True loads immediately)
    ts_from_dask = TimeSeries.from_dask(dask_arr, t0=ts.t0, dt=ts.dt, unit=ts.unit)
    print("Recovered from Dask:", ts_from_dask)

    del dask_arr, ts_from_dask

except ImportError:
    print("Dask not installed.")


--- Converted to Dask Array ---
dask.array<array, shape=(1000,), dtype=float64, chunksize=(1000,), chunktype=numpy.ndarray>
Recovered from Dask: TimeSeries([ 0.        ,  0.06279052,  0.12533323, ...,
            -0.18738131, -0.12533323, -0.06279052]
           unit: V,
           t0: 1234567890.0 1 / Hz,
           dt: 0.01 1 / Hz,
           name: None,
           channel: None)


### 1.6. JSON / Python Dict (to_json) <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **JSON**: JSON (JavaScript Object Notation) is a lightweight data exchange format supported by the Python standard library.
> 📚 [JSON Documentation](https://docs.python.org/3/library/json.html)

You can output JSON-compatible dictionary format with `to_json()` or `to_dict()`.

In [8]:
import json

# TimeSeries -> JSON string
ts_json = ts.to_json()
print("--- JSON Representation (Partial) ---")
print(ts_json[:500] + "...")

# Plot data by loading back from JSON
ts_dict_temp = json.loads(ts_json)
plt.figure()
plt.plot(ts_dict_temp["data"])
plt.title("Plot from JSON Data")
plt.show()

# Recover from JSON
from gwexpy.timeseries import TimeSeries

ts_recovered = TimeSeries.from_json(ts_json)
print("Recovered from JSON:", ts_recovered)

del ts_json, ts_dict_temp, ts_recovered

## 2. Database & Storage Layer

### 2.1. HDF5 <span style="color: #3498db; font-weight: bold;">[Original GWpy]</span>

> **HDF5**: HDF5 is a hierarchical data format for efficiently storing and managing large-scale scientific data. It can be accessed from Python via the h5py library.
> 📚 [HDF5 Documentation](https://www.h5py.org/)

Supports saving to HDF5 with `to_hdf5_dataset()`.

In [9]:
try:
    import tempfile

    import h5py

    with tempfile.NamedTemporaryFile(suffix=".h5") as tmp:
        # TimeSeries -> HDF5
        with h5py.File(tmp.name, "w") as f:
            ts.to_hdf5_dataset(f, "dataset_01")

        # Read back and display
        with h5py.File(tmp.name, "r") as f:
            ds = f["dataset_01"]
            print("--- HDF5 Dataset Info ---")
            print(f"Shape: {ds.shape}, Dtype: {ds.dtype}")
            print("Attributes:", dict(ds.attrs))

            # Recover
            from gwexpy.timeseries import TimeSeries

            ts_recovered = TimeSeries.from_hdf5_dataset(f, "dataset_01")
            print("Recovered from HDF5:", ts_recovered)

            ts_recovered.plot()
            plt.title("Recovered from HDF5")
            plt.show()

            del ds, ts_recovered

except ImportError:
    print("h5py not installed.")

### 2.2. SQLite <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **SQLite**: SQLite is a lightweight embedded SQL database engine included in the Python standard library.
> 📚 [SQLite Documentation](https://docs.python.org/3/library/sqlite3.html)

Supports database persistence with `to_sqlite()`.

In [10]:
import sqlite3

conn = sqlite3.connect(":memory:")

# TimeSeries -> SQLite
series_id = ts.to_sqlite(conn, series_id="test_series")
print(f"Saved to SQLite with ID: {series_id}")

# Verify data in SQL
cursor = conn.cursor()
row = cursor.execute("SELECT * FROM series WHERE series_id=?", (series_id,)).fetchone()
print("Metadata from SQL:", row)

# Recover
from gwexpy.timeseries import TimeSeries

ts_recovered = TimeSeries.from_sqlite(conn, series_id)
print("Recovered from SQLite:", ts_recovered)

ts_recovered.plot()
plt.title("Recovered from SQLite")
plt.show()

del series_id, conn, cursor, ts_recovered

### 2.3. Zarr <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **Zarr**: Zarr is a storage format for chunked, compressed multidimensional arrays with excellent cloud storage integration.
> 📚 [Zarr Documentation](https://zarr.dev/)

Supports cloud storage-friendly formats with `to_zarr()`.

In [11]:
try:
    import os
    import tempfile

    import zarr

    with tempfile.TemporaryDirectory() as tmpdir:
        store_path = os.path.join(tmpdir, "test.zarr")
        # TimeSeries -> Zarr
        ts.to_zarr(store_path, path="timeseries")

        # Read back
        z = zarr.open(store_path, mode="r")
        ds = z["timeseries"]
        print("--- Zarr Array Info ---")
        print(ds.info)

        # Recover
        from gwexpy.timeseries import TimeSeries

        ts_recovered = TimeSeries.from_zarr(store_path, "timeseries")
        print("Recovered from Zarr:", ts_recovered)

        ts_recovered.plot()
        plt.title("Recovered from Zarr")
        plt.show()

        del z, ds, ts_recovered

except ImportError:
    print("zarr not installed.")

### 2.4. netCDF4 <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **netCDF4**: netCDF4 is a standard format for meteorological, oceanographic, and earth science data with self-describing data structures.
> 📚 [netCDF4 Documentation](https://unidata.github.io/netcdf4-python/)

Supports meteorological and oceanographic data standards with `to_netcdf4()`.

In [12]:
try:
    import tempfile

    import netCDF4

    with tempfile.NamedTemporaryFile(suffix=".nc") as tmp:
        # TimeSeries -> netCDF4
        with netCDF4.Dataset(tmp.name, "w") as ds:
            ts.to_netcdf4(ds, "my_signal")

        # Read back
        with netCDF4.Dataset(tmp.name, "r") as ds:
            v = ds.variables["my_signal"]
            print("--- netCDF4 Variable Info ---")
            print(v)

            # Recover
            from gwexpy.timeseries import TimeSeries

            ts_recovered = TimeSeries.from_netcdf4(ds, "my_signal")
            print("Recovered from netCDF4:", ts_recovered)

            ts_recovered.plot()
            plt.title("Recovered from netCDF4")
            plt.show()

            del v, ts_recovered

except ImportError:
    print("netCDF4 not installed.")

## 3. Computer Science, Machine Learning & Accelerated Computing

### 3.1. PyTorch Integration <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **PyTorch**: PyTorch is a deep learning framework supporting dynamic computation graphs and GPU acceleration.
> 📚 [PyTorch Documentation](https://pytorch.org/)

For deep learning preprocessing, you can directly convert `TimeSeries` to `torch.Tensor`. GPU transfer is also supported.

In [13]:
import os

os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")

try:
    import torch

    # Convert to PyTorch Tensor
    tensor = ts.to_torch(dtype=torch.float32)
    print("\n--- Converted to PyTorch Tensor ---")
    print(f"Tensor shape: {tensor.shape}, dtype: {tensor.dtype}")

    # Restore from Tensor (t0, dt must be specified separately)
    ts_torch = TimeSeries.from_torch(tensor, t0=ts.t0, dt=ts.dt, unit="V")
    print("\n--- Restored from Torch ---")
    print(ts_torch)

    del tensor, ts_torch

except ImportError:
    print("PyTorch is not installed.")


--- Converted to PyTorch Tensor ---
Tensor shape: torch.Size([1000]), dtype: torch.float32

--- Restored from Torch ---
TimeSeries([ 0.        ,  0.06279052,  0.12533323, ...,
            -0.18738131, -0.12533323, -0.06279052]
           unit: V,
           t0: 1234567890.0 1 / Hz,
           dt: 0.01 1 / Hz,
           name: None,
           channel: None)


### 3.2. CuPy (CUDA Acceleration) <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **CuPy**: CuPy is a NumPy-compatible GPU array library that enables high-speed computation using NVIDIA CUDA.
> 📚 [CuPy Documentation](https://cupy.dev/)

You can convert to CuPy arrays for GPU-based computation.

In [14]:
from gwexpy.interop import is_cupy_available

if is_cupy_available():
    import cupy as cp

    # TimeSeries -> CuPy
    y_gpu = ts.to_cupy()
    print("--- CuPy Array (on GPU) ---")
    print(y_gpu)

    # Simple processing on GPU
    y_gpu_filt = y_gpu * 2.0

    # Plot (must move to CPU for plotting)
    plt.figure()
    plt.plot(cp.asnumpy(y_gpu_filt))
    plt.title("CuPy Data (Moved to CPU for plot)")
    plt.show()

    # Recover
    from gwexpy.timeseries import TimeSeries

    ts_recovered = TimeSeries.from_cupy(y_gpu_filt, t0=ts.t0, dt=ts.dt)
    print("Recovered from CuPy:", ts_recovered)

    del y_gpu, y_gpu_filt, ts_recovered

else:
    print("CuPy or CUDA driver not available.")

### 3.3. TensorFlow Integration <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **TensorFlow**: TensorFlow is a machine learning platform developed by Google, excelling at large-scale production environments.
> 📚 [TensorFlow Documentation](https://www.tensorflow.org/)

In [15]:
try:
    import os
    import warnings
    os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")
    warnings.filterwarnings("ignore", category=UserWarning, module=r"google\.protobuf\..*")
    warnings.filterwarnings("ignore", category=UserWarning, message=r"Protobuf gencode version.*")
    import tensorflow as tf

    _ = tf
    # Convert to TensorFlow Tensor
    tf_tensor = ts.to_tensorflow()
    print("\n--- Converted to TensorFlow Tensor ---")
    print(f"Tensor shape: {tf_tensor.shape}")
    print(f"Tensor dtype: {tf_tensor.dtype}")

    # Restore
    ts_from_tensorflow = TimeSeries.from_tensorflow(
        tf_tensor, t0=ts.t0, dt=ts.dt, unit=ts.unit
    )
    print("Recovered from TF:", ts_from_tensorflow)

    del tf_tensor, ts_from_tensorflow

except ImportError:
    print("TensorFlow not installed.")


--- Converted to TensorFlow Tensor ---
Tensor shape: (1000,)
Tensor dtype: <dtype: 'float64'>
Recovered from TF: TimeSeries([ 0.        ,  0.06279052,  0.12533323, ...,
            -0.18738131, -0.12533323, -0.06279052]
           unit: V,
           t0: 1234567890.0 1 / Hz,
           dt: 0.01 1 / Hz,
           name: None,
           channel: None)


### 3.4. JAX Integration <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **JAX**: JAX is a high-performance numerical computing library developed by Google, featuring automatic differentiation and XLA compilation for acceleration.
> 📚 [JAX Documentation](https://jax.readthedocs.io/)

In [16]:
try:
    import os

    os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
    import jax

    _ = jax
    import jax.numpy as jnp

    _ = jnp
    # Convert to JAX Array
    jax_arr = ts.to_jax()
    print("\n--- Converted to JAX Array ---")
    print(f"Array shape: {jax_arr.shape}")

    # Restore
    ts_from_jax = TimeSeries.from_jax(jax_arr, t0=ts.t0, dt=ts.dt, unit=ts.unit)
    print("Recovered from JAX:", ts_from_jax)

    del jax_arr, ts_from_jax
except ImportError:
    print("JAX not installed.")


--- Converted to JAX Array ---
Array shape: (1000,)
Recovered from JAX: TimeSeries([ 0.        ,  0.06279052,  0.12533323, ...,
            -0.18738131, -0.12533323, -0.06279052]
           unit: V,
           t0: 1234567890.0 1 / Hz,
           dt: 0.01 1 / Hz,
           name: None,
           channel: None)


## 4. Astronomy & Gravitational Wave Physics

### 4.1. PyCBC / LAL <span style="color: #3498db; font-weight: bold;">[Original GWpy]</span>

> **LAL**: LAL (LIGO Algorithm Library) is the official analysis library for LIGO/Virgo, providing the foundation for gravitational wave analysis.
> 📚 [LAL Documentation](https://lscsoft.docs.ligo.org/lalsuite/)

> **PyCBC**: PyCBC is a library for gravitational wave data analysis, used for signal searches and parameter estimation.
> 📚 [PyCBC Documentation](https://pycbc.org/)

Provides compatibility with standard gravitational wave analysis tools.

### 4.2. Astropy Integration <span style="color: #3498db; font-weight: bold;">[Original GWpy]</span>

> **Astropy**: Astropy is a Python library for astronomy, supporting coordinate transformations, time system conversions, unit systems, and more.
> 📚 [Astropy Documentation](https://www.astropy.org/)

Also supports interconversion with `astropy.timeseries.TimeSeries`, which is standard in the astronomy field.

In [17]:
try:
    # Convert to Astropy TimeSeries
    ap_ts = ts.to_astropy_timeseries()
    print("\n--- Converted to Astropy TimeSeries ---")
    print(ap_ts[:5])
    fig, ax = plt.subplots()
    ax.plot(ap_ts.time.jd, ap_ts["value"])
    plt.title("Astropy TimeSeries")
    plt.show()
    plt.close()

    # Restore
    ts_astro = TimeSeries.from_astropy_timeseries(ap_ts)
    print("\n--- Restored from Astropy ---")
    print(ts_astro)

    del ap_ts, ts_astro

except ImportError:
    print("Astropy is not installed.")

### 4.3. Specutils Integration <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **specutils**: specutils is an Astropy-affiliated package for manipulating and analyzing astronomical spectral data.
> 📚 [specutils Documentation](https://specutils.readthedocs.io/)

`FrequencySeries` can interconvert with `Spectrum1D` objects from the Astropy ecosystem spectral analysis library `specutils`.
Units and frequency axes are properly preserved.

In [18]:
try:
    import specutils

    _ = specutils

    from gwexpy.frequencyseries import FrequencySeries

    # FrequencySeries -> specutils.Spectrum1D
    fs = FrequencySeries(
        np.random.random(100), frequencies=np.linspace(10, 100, 100), unit="Jy"
    )
    spec = fs.to_specutils()
    print("specutils Spectrum1D:", spec)
    print("Spectral axis unit:", spec.spectral_axis.unit)

    # specutils.Spectrum1D -> FrequencySeries
    fs_rec = FrequencySeries.from_specutils(spec)
    print("Restored FrequencySeries unit:", fs_rec.unit)

    del fs, spec, fs_rec

except ImportError:
    print("specutils library not found. Skipping example.")

specutils Spectrum1D: Spectrum (length=100)
Flux=[0.76947961 0.77852583 0.20217518 ... 0.73100724 0.63882659
      0.18566241] Jy,  mean=0.43060 Jy
Spectral Axis=[ 10.          10.90909091  11.81818182 ...
                98.18181818  99.09090909 100.        ] Hz,  mean=55.00000 Hz
Spectral axis unit: Hz
Restored FrequencySeries unit: Jy


### 4.4. Pyspeckit Integration <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **PySpecKit**: PySpecKit is a spectral analysis toolkit for radio astronomy, supporting spectral line fitting and more.
> 📚 [PySpecKit Documentation](https://pyspeckit.readthedocs.io/)

`FrequencySeries` can also integrate with `Spectrum` objects from the general-purpose spectral analysis toolkit `pyspeckit`.

In [19]:
try:
    import pyspeckit

    _ = pyspeckit
    from gwexpy.frequencyseries import FrequencySeries

    # FrequencySeries -> pyspeckit.Spectrum
    fs = FrequencySeries(np.random.random(100), frequencies=np.linspace(10, 100, 100))
    spec = fs.to_pyspeckit()
    print("pyspeckit Spectrum length:", len(spec.data))

    # pyspeckit.Spectrum -> FrequencySeries
    fs_rec = FrequencySeries.from_pyspeckit(spec)
    print("Restored FrequencySeries length:", len(fs_rec))

    del fs, spec, fs_rec

except ImportError:
    print("pyspeckit library not found. Skipping example.")

pyspeckit library not found. Skipping example.


## 5. Particle Physics & High Energy Physics

### 5.1. CERN ROOT Integration <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **ROOT**: ROOT is a data analysis framework for high-energy physics developed by CERN.
> 📚 [ROOT Documentation](https://root.cern/)

Enhanced interoperability with **ROOT**, the standard tool in high-energy physics.
`gwexpy` Series objects can be quickly converted to ROOT's `TGraph`, `TH1D`, or `TH2D`, and you can create ROOT files.
Conversely, you can also restore `TimeSeries` and other objects from ROOT objects.

**Note**: Using this feature requires `ROOT` (PyROOT) to be installed.

In [20]:
try:
    import numpy as np
    import ROOT

    from gwexpy.timeseries import TimeSeries

    # Prepare data
    t = np.linspace(0, 10, 1000)
    data = np.sin(2 * np.pi * 1.0 * t) + np.random.normal(0, 0.5, size=len(t))
    ts = TimeSeries(data, dt=t[1] - t[0], name="signal")

    # --- 1. Convert to TGraph ---
    # Vectorized high-speed conversion
    graph = ts.to_tgraph()

    # Plot on ROOT canvas
    c1 = ROOT.TCanvas("c1", "TGraph Example", 800, 600)
    graph.SetTitle("ROOT TGraph;GPS Time [s];Amplitude")
    graph.Draw("AL")
    c1.Draw()
    # c1.SaveAs("signal_graph.png") # To save as an image

    print(f"Created TGraph: {graph.GetName()} with {graph.GetN()} points")

    # --- 2. Convert to TH1D (Histogram) ---
    # Convert as histogram (binning is automatic or can be specified)
    hist = ts.to_th1d()

    c2 = ROOT.TCanvas("c2", "TH1D Example", 800, 600)
    hist.SetTitle("ROOT TH1D;GPS Time [s];Amplitude")
    hist.SetLineColor(ROOT.kRed)
    hist.Draw()
    c2.Draw()

    print(f"Created TH1D: {hist.GetName()} with {hist.GetNbinsX()} bins")

    del t, data, graph, hist, c1, c2

except ImportError:
    print("ROOT is not installed. Skipping ROOT examples.")
except Exception as e:
    print(f"An error occurred: {e}")

Created TGraph: signal with 1000 points
Created TH1D: signal with 1000 bins


### 5.2. Recovering from ROOT Objects (`from_root`)

> **ROOT**: ROOT is a data analysis framework for high-energy physics developed by CERN.
> 📚 [ROOT Documentation](https://root.cern/)

You can load histograms and graphs from existing ROOT files and convert them back to `gwexpy` objects for easier analysis.

In [21]:
try:
    if "hist" in locals() and hist:
        # ROOT TH1D -> TimeSeries
        # Reads histogram bin contents as time series data
        ts_restored = from_root(TimeSeries, hist)

        print(f"Restored TimeSeries: {ts_restored.name}")
        print(ts_restored)

        # Restore from TGraph similarly
        ts_from_graph = from_root(TimeSeries, graph)
        print(f"Restored from TGraph: {len(ts_from_graph)} samples")

        del ts_restored, ts_from_graph

except NameError:
    pass  # If hist or graph were not created
except ImportError:
    pass

## 6. Geophysics, Seismology & Electromagnetics

### 6.1. ObsPy <span style="color: #3498db; font-weight: bold;">[Original GWpy]</span>

> **ObsPy**: ObsPy is a Python library for acquiring, processing, and analyzing seismological data, supporting formats like MiniSEED.
> 📚 [ObsPy Documentation](https://docs.obspy.org/)

Supports interoperability with ObsPy, which is standard in seismology.

In [22]:
try:
    import obspy

    _ = obspy
    # TimeSeries -> ObsPy Trace
    tr = ts.to_obspy()
    print("--- ObsPy Trace ---")
    print(tr)

    # Plot using ObsPy
    tr.plot()

    # Recover to TimeSeries
    from gwexpy.timeseries import TimeSeries

    ts_recovered = TimeSeries.from_obspy(tr)
    print("Recovered from ObsPy:", ts_recovered)

    del tr, ts_recovered

except ImportError:
    print("ObsPy not installed.")

In [23]:
try:
    import obspy

    _ = obspy
    tr = ts.to_obspy()
    print("ObsPy Trace:", tr)

    del tr

except ImportError:
    print("ObsPy not installed.")

ObsPy Trace: .signal.. | 1980-01-06T00:00:00.000000Z - 1980-01-06T00:00:10.000000Z | 99.9 Hz, 1000 samples


### 6.2. SimPEG Integration <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **SimPEG**: SimPEG is a simulation and estimation framework for geophysical inverse problems.
> 📚 [SimPEG Documentation](https://simpeg.xyz/)

`gwexpy` supports integration with `SimPEG`, a forward modeling and inversion library for geophysics.
`TimeSeries` (TDEM) and `FrequencySeries` (FDEM) can be converted to `simpeg.data.Data` objects and vice versa.

In [24]:
try:
    import numpy as np
    import simpeg

    _ = simpeg
    from simpeg import maps

    _ = maps

    from gwexpy.frequencyseries import FrequencySeries
    from gwexpy.timeseries import TimeSeries

    # --- TimeSeries -> SimPEG (TDEM assumption) ---
    ts = TimeSeries(np.random.normal(size=100), dt=0.01, unit="A/m^2")
    simpeg_data_td = ts.to_simpeg(location=np.array([0, 0, 0]))
    print("SimPEG TDEM data shape:", simpeg_data_td.dobs.shape)

    # --- FrequencySeries -> SimPEG (FDEM assumption) ---
    fs = FrequencySeries(
        np.random.normal(size=10) + 1j * 0.1, frequencies=np.logspace(0, 3, 10)
    )
    simpeg_data_fd = fs.to_simpeg(location=np.array([0, 0, 0]), orientation="z")
    print("SimPEG FDEM data shape:", simpeg_data_fd.dobs.shape)

    del simpeg_data_td, simpeg_data_fd, fs

except ImportError:
    print("SimPEG library not found. Skipping example.")

SimPEG TDEM data shape: (100,)
SimPEG FDEM data shape: (10,)


### 6.3. MTH5 / MTpy <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **MTH5**: MTH5 is an HDF5-based data format for magnetotelluric (MT) data.
> 📚 [MTH5 Documentation](https://mth5.readthedocs.io/)

Supports MTH5 storage for magnetotelluric method data.

In [25]:
try:
    import tempfile

    import mth5

    _ = mth5
    with tempfile.NamedTemporaryFile(suffix=".h5") as tmp:
        from gwexpy.interop.mt_ import from_mth5, to_mth5

        # TimeSeries -> MTH5
        # We need to provide station and run names for MTH5 structure
        to_mth5(ts, tmp.name, station="SITE01", run="RUN01")
        print(f"Saved to MTH5 file: {tmp.name}")

        # Display structure info if needed, or just recover
        # Recover
        ts_recovered = from_mth5(tmp.name, "SITE01", "RUN01", ts.name or "Ex")
        print("Recovered from MTH5:", ts_recovered)

        ts_recovered.plot()
        plt.title("Recovered from MTH5")
        plt.show()

        del ts_recovered

except ImportError:
    print("mth5 not installed.")

## 7. Acoustics & Audio Analysis

### 7.1. Librosa / Pydub <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **pydub**: pydub is a simple library for audio file manipulation (editing, conversion, effects).
> 📚 [pydub Documentation](https://github.com/jiaaro/pydub)

> **librosa**: librosa is a library for audio and music analysis, providing features like spectral analysis and beat detection.
> 📚 [librosa Documentation](https://librosa.org/)

Supports integration with audio processing libraries Librosa and Pydub.

In [26]:
try:
    import librosa

    _ = librosa
    import matplotlib.pyplot as plt

    # TimeSeries -> Librosa (y, sr)
    y, sr = ts.to_librosa()
    print(f"--- Librosa Data ---\nSignal shape: {y.shape}, Sample rate: {sr}")

    # Plot using librosa style (matplotlib)
    plt.figure()
    plt.plot(y[:1000])  # Plot first 1000 samples
    plt.title("Librosa Audio Signal (Zoom)")
    plt.show()

    # Recover to TimeSeries
    from gwexpy.timeseries import TimeSeries

    ts_recovered = TimeSeries(y, dt=1.0 / sr)
    print("Recovered from Librosa:", ts_recovered)

    del y, sr
except ImportError:
    print("Librosa not installed.")

## 8. Medical & Biosignal Analysis

### 8.1. MNE-Python <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **MNE**: MNE-Python is a library for analyzing EEG and MEG data, widely used in neuroscience research.
> 📚 [MNE Documentation](https://mne.tools/)

Integration with MNE for electroencephalography (EEG/MEG) analysis packages.

In [27]:
try:
    import mne

    _ = mne
    # TimeSeries -> MNE Raw
    raw = ts.to_mne()
    print("--- MNE Raw ---")
    print(raw)

    # Display info
    print(raw.info)

    # Recover to TimeSeries
    from gwexpy.timeseries import TimeSeries

    ts_recovered = TimeSeries.from_mne(raw, channel=ts.name or "ch0")
    print("Recovered from MNE:", ts_recovered)

    del raw, ts_recovered

except ImportError:
    print("MNE not installed.")

In [28]:
try:
    import mne

    _ = mne
    raw = ts.to_mne()
    print("MNE Raw:", raw)

    del raw
except ImportError:
    print("MNE not installed.")

### 8.2. Elephant / quantities Integration <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

`gwexpy`'s `FrequencySeries` and `Spectrogram` can interconvert with `quantities.Quantity` objects.
This is useful for integration with Elephant and Neo.

Note: Requires `pip install quantities` beforehand.

In [29]:
try:
    import numpy as np
    import quantities as pq

    _ = pq
    from gwexpy.frequencyseries import FrequencySeries

    # Create FrequencySeries
    freqs = np.linspace(0, 100, 101)
    data_fs = np.random.random(101)
    fs = FrequencySeries(data_fs, frequencies=freqs, unit="V")

    # to_quantities
    q_obj = fs.to_quantities(units="mV")
    print("Quantities object:", q_obj)

    # from_quantities
    fs_new = FrequencySeries.from_quantities(q_obj, frequencies=freqs)
    print("Restored FrequencySeries unit:", fs_new.unit)

    del freqs, data_fs, fs, q_obj, fs_new

except ImportError:
    print("quantities library not found. Skipping example.")

Quantities object: [ 15.81608925 752.98483078  24.72547363 144.51628594
 214.32932705 200.42267588 739.82568485 135.47985311
 640.01545789 559.50533316   5.69614253 200.02784218
 768.04102342 903.95362763 395.33110766 489.09599556
 180.3017241  426.32334648 574.85398845 820.69910971
 964.17971075 578.39473516 212.14270287 142.49981321
 731.86090217 311.377196   557.94520695 584.93105318
  40.80640136 631.68944016 836.03253181 329.13934202
 402.7914657   73.93266668  61.06459887 866.72322773
 453.00444668 497.81654835 576.19232129 390.48186662
  88.95723072 197.64563216 125.14799417 635.59725698
 435.48720445 463.36157347 718.48503157 943.63542519
  91.65132826 320.40017851 528.74706589 288.93260417
 683.23507876 606.7956871  413.8774318  250.05522479
 209.12606513 497.85663591 635.85855813 130.81356845
 707.52091187 130.21639147 325.76429004 952.1717244
 707.27537209 872.99294786 246.98162862 809.39348485
 722.67100602 372.64538896 222.55272929 335.34166876
 809.63936761 798.69696411  

### 8.3. Neo <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **Neo**: Neo is a data structure library for electrophysiology data (neuroscience), supporting input/output to various formats.
> 📚 [Neo Documentation](https://neo.readthedocs.io/)

Supports conversion to Neo, a common standard for electrophysiological data.

In [30]:
try:
    import neo

    _ = neo
    # TimeSeries -> Neo AnalogSignal
    # to_neo is available in gwexpy.interop
    # Note: TimeseriesMatrix is preferred for multi-channel Neo conversion,
    # but we can convert single TimeSeries by wrapping it.
    from gwexpy.interop import from_neo, to_neo

    _ = from_neo
    _ = to_neo
    # For single TimeSeries, we might need a Matrix wrapper or direct helper.
    # Assuming helper exists or using Matrix:
    from gwexpy.timeseries import TimeSeriesMatrix

    tm = TimeSeriesMatrix(
        ts.value[None, None, :], t0=ts.t0, dt=ts.dt, channel_names=[ts.name]
    )
    sig = tm.to_neo()

    print("--- Neo AnalogSignal ---")
    print(sig)

    # Display/Plot
    plt.figure()
    plt.plot(sig.times, sig)
    plt.title("Neo AnalogSignal Plot")
    plt.show()

    # Recover
    tm_recovered = TimeSeriesMatrix.from_neo(sig)
    ts_recovered = tm_recovered[0]
    print("Recovered from Neo:", ts_recovered)

    del tm, tm_recovered, sig, ts_recovered

except ImportError:
    print("neo not installed.")

## 9. Summary

`gwexpy` provides interoperability with a wide variety of domain-specific libraries, enabling seamless integration with existing ecosystems.